### Install required packages

In [ ]:
!pip install google-generativeai

### Import required libraries

In [2]:
import google.generativeai as palm
import csv
import time
import pandas as pd
from datetime import datetime
import os
from google.colab import userdata
import json

###Set the API KEY using Google colab user data option

In [3]:
palm_access = userdata.get('palm')
os.environ['GOOGLE_API_KEY'] = palm_access

###Read the configurations

In [4]:
json_file_path = "/content/drive/MyDrive/Colab Notebooks/config.json"
with open(json_file_path, "r") as file:
    configuration = json.load(file)

### Load the user story descriptions to data frame

In [5]:
input_csv_path = configuration['input_csv_path']
# Read the complete CSV file as a pandas DataFrame
data_frame = pd.read_csv(input_csv_path)


## STEPS
### 1.   Read each user story description
### 2.   Set the model [chat-bison-01] and model parameters
### 3.   Read the zero, few-shot prompt from configuration
### 4.   call Palm chat completion end point
### 5. save the response as .feature file

In [ ]:
# Specify the columns to iterate
#step 1
us_desc_column=configuration['us_desc_column']
app_desc_column=configuration['app_desc_column']
# read model, configuration parameters
#step 2
palm_model=configuration['palm_model']
palm_temperature=configuration['palm_temperature']
palm_top_p=configuration['palm_top_p']
palm_max_tokens=configuration['palm_max_tokens']
response_file_name_palm=configuration['response_feature_file_palm']
few_shot_method=configuration['few_shot_method']
zero_shot_method=configuration['zero_shot_method']
output_csv_path=configuration['output_csv_path']
#step 3
system_prompt=configuration['system_prompt']
for index, row in data_frame.iterrows():
    user_story = row[us_desc_column]
    story_context = row[app_desc_column]
    #step 3
    zero_shot_array=configuration['zero_shot']
    zero_shot=' '.join(zero_shot_array)
    few_shot_user_prompt_array=configuration['few_shot_user_prompt']
    few_shot_user_prompt=' '.join(few_shot_user_prompt_array)
    few_shot_assistant_prompt_array=configuration['few_shot_assistant_prompt']
    few_shot_assistant_prompt=' '.join(few_shot_assistant_prompt_array)
    examples=[
    (few_shot_user_prompt, few_shot_assistant_prompt),
    ]
    user_message = zero_shot + user_story
    #step 4
    palm.configure()
    response=palm.chat(messages=user_message, temperature=palm_temperature,
            top_p=palm_top_p,
            model=palm_model,
           # max_tokens=palm_max_tokens,
            examples=examples,
            context=system_prompt)
     # Extract the response
    for message in response.messages:
          if message['author'] == '1':
            author_1_content = message['content']
    output_text = author_1_content
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    #step 5
    response_file_name = f"US{index}_{timestamp}_{few_shot_method}_{response_file_name_palm}.feature"
    #step
    # step 5
    feature_file_path = output_csv_path+response_file_name
    with open(feature_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(output_text)
    time.sleep(20)
    # Write the output texts to a text file
